# Holocene Great Barrier Reef evolution


At basin-scale sediment transport and reef evolution are strongly controlled by large-scale forcings. Our model allows consideration of the following set of external forcing mechanisms: 
+ sea-level oscillations, 
+ rates of tectonic changes, 
+ rainfall regimes and 
+ boundary wave conditions. 

Spatial and temporal variations in precipitation can be applied as a set of maps representing an annual rainfall regime. The tectonic changes are provided as a series of temporal maps. Each map can have variable spatial cumulative displacements making it possible to simulate complex 3D tectonic evolution with both vertical (uplift and subsidence) and horizontal directions. The combination of these forcing mechanisms controls the evolution of the hydrodynamic conditions and the associated sediment transport regimes as well as marine carbonate production.

***

<img src="images/erodep.jpg" alt="geometry" width="1000" height="500"/>

***

This example is based on the following paper:

**Salles T., X. Ding, J.M. Webster, A. Vila-Concejo, G. Brocard and J. Pall: A unified framework for modelling sediment fate from source to sink and its interactions with reef systems over geological times, Scientific Report, [doi:10.1038/s41598-018-23519-8](https://doi.org/10.1038/s41598-018-23519-8), 2018.**

***

Maps of cumulative erosion/deposition in metres at final time step combining fluvial, waves and coral reef processes for northern (left) and southern (right) GBR regions - for higher resolution simulation than the one propose in this example.

In [ ]:
import glob
import lavavu
import numpy as np
import cmocean as cmo
from matplotlib import cm

from scripts import viewBadlands as visu
from scripts import sectionCarbonate as section
from scripts import analyseWaveReef as wrAnalyse

from badlands.model import Model as badlandsModel

# display plots in SVG format
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

## Starting badlands

First, we initialise the model and set the path to the **XmL** input file:

+ **input.xml**

> You can edit the **XmL** configuration file directly in the _Jupyter environment_ by clicking on it in the `tree` and changing in the **url** the **view** to **edit**.

In [ ]:
# Initialise model
model = badlandsModel()

# Define the XmL input file
model.load_xml('input.xml')

## Running badlands

We can run the model for a given period. The end time in the **XmL** input file is set to _0_ (present day) but you might want to run the model for a coupled of iterations and check the output before running the model for the entire simulation time. 

This is done by putting the time in the `run_to_time` function. 

Here we go for the full time directly...

In [ ]:
model.run_to_time(0)

## Visualise outputs

In [ ]:
folder = 'output'
stepCounter = len(glob.glob1(folder+"/xmf/","tin.time*"))-1
print("Number of visualisation steps created: ",stepCounter)
visu.viewTime(folder, steps=stepCounter, it=1, scaleZ=10, maxZ=3000, maxED=20, maxwED=10)

## Extracting model outputs

***

<img src="images/wavegbr.jpg" alt="geometry" width="800" height="550"/>

*** 

<div class="alert alert-block alert-info">We will extract different parameters from our simulation and do some basic post-processing analyses.</div>

Some of the things that could be explore:

+ how high sediment loads from catchments erosion influenced coral growth during the transgression phase,
+ what is the relation between inland incision and sediment gravity-flows in the deepest parts of the basin floor; 
+ what are the relationships between climate, sea-level, and margin physiography and how it enabled coral reefs to thrive.

We can plot a series of different parameters from the model for the chosen timestep:

+ elevation: `dataTIN.z`
+ slope: `dataTIN.slp`
+ total erosion/deposition: `dataTIN.dz`
+ wave-induced erosion/deposition: `dataTIN.wavesed`
+ wave-induced shear stress: `dataTIN.wavestress`
+ hillslope induced erosion/deposition : `dataTIN.cumhill`
+ reef present/absent: `dataTIN.depreef`
+ aspect: `dataTIN.aspect`
+ horizontal curvature: `dataTIN.hcurv`
+ vertical curvature: `dataTIN.vcurv`

In [ ]:
dataTIN = wrAnalyse.analyseWaveReef(folder='output',timestep=22)
dataTIN.regridTINdataSet()

dataTIN.plotdataSet(title='Elevation', data=dataTIN.z, color=cmo.cm.delta,crange=[-2000,2000])
dataTIN.plotdataSet(title='Total erosion/deposition [m]', data=dataTIN.dz, color=cmo.cm.balance,crange=[-20.,20.])
dataTIN.plotdataSet(title='Wave erosion/deposition [m]', data=dataTIN.wavesed, color=cmo.cm.balance,crange=[-5.,5.])
dataTIN.plotdataSet(title='Hillslope erosion/deposition [m]', data=dataTIN.cumhill, color=cmo.cm.balance,
                      crange=[-2.5,2.5])
dataTIN.plotdataSet(title='Wave induced stress', data=dataTIN.wavestress, color=cmo.cm.tempo,crange=[0.,2.],ctr='k')
dataTIN.plotdataSet(title='Reef absent/present', data=dataTIN.depreef, color=cm.Blues,
                      crange=[0,2])

## Analysing sediment erosion/deposition and reef growth

We have access to the deposited volume over time and can perform with IPython notebook calculation of the accumulated volume in the entire region or in a specific area. This can be done in this way:


We will use the following area for this example:

***

<img src="images/geo.jpg" alt="geometry" width="800" height="550"/>

*** 


We first define the area where we will perform our analyse:

+ `erange`: [100000,250000,540000,640000]

and plot the total sedimentation change (erosion & deposition) `dataTIN.dz`.

In [ ]:
dataTIN.plotdataSet(title='Erosion/Deposition [m]', data=dataTIN.dz, color=cmo.cm.balance,  
                      crange=[-25,25], erange=[100000,250000,540000,640000],
                      depctr=(-2,10),ctr='k',size=(10,10))

Then we look if there is any reef in the region... we should have some based on the GoogleMap image above!

In [ ]:
dataTIN.plotdataSet(title='Reef position', data=dataTIN.depreef, color=cmo.cm.balance,  
                      crange=[-2,2], erange=[100000,250000,540000,640000],
                      depctr=(1),ctr='k',size=(10,10))

We can now compute a series of information regarding erosion/deposition volume for the selected region.

#### Total volume of erosion and deposition

In [ ]:
dataTIN.getDepositedVolume(data=dataTIN.dz,time=11000.,erange=[100000,250000,540000,640000])
print('----')
dataTIN.getErodedVolume(data=dataTIN.dz,time=11000.,erange=[100000,250000,540000,640000])

#### Total volume of erosion and deposition induced by hillslope processes

In [ ]:
dataTIN.getDepositedVolume(data=dataTIN.cumhill,time=11000.,erange=[100000,250000,540000,640000])
print('----')
dataTIN.getErodedVolume(data=dataTIN.cumhill,time=11000.,erange=[100000,250000,540000,640000])

#### Total volume of erosion and deposition induced by wave processes

In [ ]:
dataTIN.getDepositedVolume(data=dataTIN.wavesed,time=11000.,erange=[100000,250000,540000,640000])
print('----')
dataTIN.getErodedVolume(data=dataTIN.wavesed,time=11000.,erange=[100000,250000,540000,640000])

#### Total volume of erosion and deposition around reef systems

In [ ]:
r,c = np.where(dataTIN.depreef>0)
erodepreef = np.zeros(dataTIN.dz.shape)
erodepreef[r,c] = dataTIN.dz[r,c]
dataTIN.getDepositedVolume(data=erodepreef,time=11000.,erange=[100000,250000,540000,640000])
print('----')
dataTIN.getErodedVolume(data=erodepreef,time=11000.,erange=[100000,250000,540000,640000])

### Temporal analyse of reef growth

Up to now, we have only looked at the last time step, obviously we have access to all these information over time in our model. 

Here is an example of how to plot deposition and erosion rate through time on the reefs only. It can give some information on the timing of the reefs _turn-on_ for example in this particular area. 

This can be done in this way:

In [ ]:
time = np.linspace(-10000.0, 0.0, num=21)
time,ero,depo = wrAnalyse.temporalGrowth(folder='output',step=22,vtime=time,
                                         smooth=100,vrange=[100000,250000,540000,640000])
wrAnalyse.temporalPlot(time,ero,depo,size=(8,5))

## Reef cross section


***

<img src="images/section.jpg" alt="geometry" width="940" height="800"/>

***

Example of cross sections through one Badlands model predicted stratigraphy showing time layers of mixed siliciclastic-carbonate accretion NW of Swain Reef and offshore of Princess Charlotte Bay.


<div class="alert alert-block alert-info">We will use the recorded underlying stratigraphy to extract information regarding reef growth in different regions of the continental shelf.</div>

### Section locations

First we will define the locations where we would like to make some cross-sections.

We define the model output `folder` in our case it is called _output_ and the time `step` we want to plot (here 22, the final time). 

We define each cross-sections based on their X,Y coordinates as _numpy_ vector (_np_).

In [ ]:
secmap = section.sectionCarbonate(folder = 'output',step=22)

# Section 1
sec1=np.zeros((2,2))
sec1[0,:] = [287882,469598]
sec1[1,:] = [368169,572448]

# Section 2
sec2=np.zeros((2,2))
sec2[0,:] = [134722,626561]
sec2[1,:] = [257105,500163]

# Section 3
sec3=np.zeros((2,2))
sec3[0,:] = [481670,62921.5]
sec3[1,:] = [603090,26405]

We then combine them together in a Python list using the command `append()`

In [ ]:
pt = []
pt.append(sec1)
pt.append(sec2)
pt.append(sec3)

Let's have a look at where our cross-sections are:

In [ ]:
secmap.plotSectionMap(title='Elevation', color=cmo.cm.delta, crange=[-5000,5000], pt=pt,size=(8,8))

<div class="alert alert-block alert-info">We will now plot each of these sections in the notebook.</div>


#### Cross section 1

In [ ]:
secCarb1 = section.sectionCarbonate(folder = 'output',step=22)
secCarb1.interpolate(dump=False)
secCarb1.buildSection(sec=sec1)
section.viewSection(width = 900, height = 700, cs = secCarb1, 
            dnlay = 1, rangeX=[30000, 98000], rangeY=[-75,40],
            linesize = 0.5, title=' ')

#### Cross section 2

In [ ]:
secCarb2 = section.sectionCarbonate(folder = 'output',step=22)
secCarb2.interpolate(dump=False)
secCarb2.buildSection(sec=sec2)
section.viewSection(width = 900, height = 700, cs = secCarb2, 
            dnlay = 1, rangeX=[0, 170000], rangeY=[-75,120],
            linesize = 0.5, title=' ')

#### Cross section 3

In [ ]:
secCarb3 = section.sectionCarbonate(folder = 'output',step=22)
secCarb3.interpolate(dump=False)
secCarb3.buildSection(sec=sec3)
section.viewSection(width = 900, height = 700, cs = secCarb3, 
            dnlay = 1, rangeX=[0, 120000], rangeY=[-75,50],
            linesize = 0.5, title=' ')